In [92]:
from bs4 import BeautifulSoup
import requests
from urllib.request import Request, urlopen
import pandas as pd
import numpy as np
%autosave 120

Autosaving every 120 seconds


In [122]:
def generateLeagueData(mainplayers):
    concatframes = pd.DataFrame()
    for mainplayer in mainplayers:
        tempdata = getLeagueData(mainplayer)
        
        if len(tempdata) != 0:
            tempdata["blue_avg_wr"] = tempdata["blue_winrates"].apply(lambda x: np.mean(x))
            tempdata["red_avg_wr"] = tempdata["red_winrates"].apply(lambda x: np.mean(x))
            tempdata["wr_difference"] = tempdata["blue_avg_wr"]-tempdata["red_avg_wr"]
        else:
            continue
        
        concatframes = pd.concat([concatframes, tempdata])
    
    return concatframes

In [123]:
def getLeagueData(mainplayer):
    site= "https://na.op.gg/summoner/userName={}".format(mainplayer)
    hdr = {'User-Agent': 'Mozilla/5.0'}
    page = requests.get(site, headers=hdr)
    soup = BeautifulSoup(page.text, "html.parser")
    #req = Request(site,headers=hdr)
    #page = urlopen(req)
    #soup = BeautifulSoup(page, "html.parser")
    
    mainplayer = mainplayer.replace(" ", "")
    
    gamemodes = soup.find_all(class_="GameType")
    
    if len(gamemodes) < 10:
        return pd.DataFrame()
    
    gamemode = []
    for i in range(len(gamemodes)):
        gamemode.append(gamemodes[i].get_text().strip())
    
    #while len(gamemode) < 10:
    #    gamemode.append("NA")
        
    gameresults = soup.find_all(class_="GameResult")
    gameresult = []
    for i in range(10):
        gameresult.append(gameresults[i].get_text().strip())
        
    #while len(gameresults) < 10
    #    gameresult.append("NA")
        
    summonernames = soup.find_all(lambda tag: tag.get('class') == ['SummonerName'])
    sumnames = []
    for i in range(len(summonernames)):
        sumnames.append(summonernames[i].get_text().strip().replace(" ", ""))
    gamechunks = [sumnames[x:x+5] for x in range(0, len(sumnames), 5)]
    
    blueteams = [gamechunks[n] for n in range(len(gamechunks)) if n % 2 == 0]
    redteams = [gamechunks[n] for n in range(len(gamechunks)) if n % 2 != 0]
    
    bluewin = []
    for i in range(len(gameresult)):
        if (mainplayer in blueteams[i] and gameresult[i] == "Victory") or (mainplayer in redteams[i] and gameresult[i] == "Defeat"):
            bluewin.append(1)
        else:
            bluewin.append(0)
        
    bluewinrates = []
    for i in range(len(blueteams)):
        bluewinrates.append(getMultiData(blueteams[i]))
    
    redwinrates = []
    for i in range(len(redteams)):
        redwinrates.append(getMultiData(redteams[i]))
    
    return pd.DataFrame({"Queue_Type": gamemode, "blue_win": bluewin, "blue_team": blueteams, "red_team": redteams,
                  "blue_winrates": bluewinrates, "red_winrates": redwinrates})

In [124]:
def getMultiData(team):
    site= "https://na.op.gg/multi/query={}%2C{}%2C{}%2C{}%2C{}".format(team[0],team[1],team[2],team[3],team[4])
    hdr = {'User-Agent': 'Mozilla/5.0'}
    page = requests.get(site, headers=hdr)
    soup = BeautifulSoup(page.text, "html.parser")
    #req = Request(site,headers=hdr)
    #page = urlopen(req)
    #soup = BeautifulSoup(page, "html.parser")  
    
    gamesplayed = soup.find_all(class_="base")
    winloss = []
    if len(gamesplayed) == 5:
        for i in range(len(team)):
            winloss.append(gamesplayed[i].get_text().strip().replace(" ", "").replace("\n", "").replace("L", "").split("W"))
    else:
        for i in range(len(gamesplayed)):
            winloss.append(gamesplayed[i].get_text().strip().replace(" ", "").replace("\n", "").replace("L", "").split("W"))
    
    while len(winloss) < 5:
        winloss.append(["1","1"])
    
    return [round(float(wr[0])/(float(wr[0])+float(wr[1])), 5) for wr in winloss]

In [86]:
real = generateLeagueData(["Warnedjacka"])

In [87]:
real

,Queue_Type,blue_win,blue_team,red_team,blue_winrates,red_winrates,blue_avg_wr,red_avg_wr,wr_difference
0,Ranked Solo,0,"[RanchesterCity, IlIIIllIIIllIIl, Dominating, ...","[NamedMyDogSusan, ZhuóJiuLoveU, Warnedjacka, R...","[0.49571, 0.54697, 0.51355, 0.50704, 0.49658]","[0.5066, 0.56463, 0.50111, 0.50873, 0.5189]",0.511970,0.519994,-0.008024
1,Ranked Solo,0,"[Warnedjacka, YuseiFudoz, TL869, ZapRowsdower,...","[AdamjDoomer, Shìmmer, Inumerable, Invitrix, P...","[0.54167, 0.57252, 0.53039, 0.50094, 0.50111]","[0.50052, 0.52599, 0.52703, 0.59774, 0.50027]",0.529326,0.530310,-0.000984
2,Ranked Solo,1,"[superpandafan37, C9YeKiCoach, Warnedjacka, Ty...","[Mataz, IceSnowClever, ThomasSheIby, twtvDanie...","[0.57668, 0.54762, 0.51565, 0.5, 0.50111]","[0.51391, 0.51688, 0.49635, 0.49554, 0.55914]",0.528212,0.516364,0.011848
3,Ranked Solo,0,"[MELLORONKLEDXD, Jimpai, Warnedjacka, SW4P, SW...","[Tuoruway, Possess, Trashopeia, Falling, Empre...","[0.50538, 0.51653, 0.50833, 0.50111, 0.52896]","[0.64341, 0.64706, 0.50918, 0.50078, 0.55743]",0.512062,0.571572,-0.059510
4,Ranked Solo,0,"[Warnedjacka, SpongeeBobb, Soulzzz, Cendilion,...","[distelfinks, Shìmmer, BlessedbyDread, UdonBdo...","[0.52901, 0.5, 0.52976, 0.50111, 0.6875]","[0.50291, 0.52599, 0.52703, 0.53125, 0.51672]",0.549476,0.520780,0.028696
5,Ranked Solo,1,"[Warnedjacka, Density, Mid2EZ, Mahusen, Arctic...","[marios, DandelionYi, Hiimcz, CrimsonX10, Urba...","[0.51675, 0.50174, 0.50047, 0.50111, 0.65574]","[0.50919, 0.5298, 0.51198, 0.5235, 0.51814]",0.535162,0.518522,0.016640
6,Ranked Solo,1,"[Shawner, Warnedjacka, Carahill, PuertoJew, fr...","[inchesevan, lucky888, DoctorYas, Bounty, SEXS...","[0.51544, 0.49655, 0.49544, 0.50111, 0.50873]","[0.51002, 0.49099, 0.49652, 0.50364, 0.49448]",0.503454,0.499130,0.004324
7,Ranked Solo,1,"[fkmyteam, ˆKrisWengˆ, K4T4, Kesslér, Millenni...","[Dowding, 08AM, Warnedjacka, ZapRowsdower, Bru...","[0.4989, 0.5, 0.5641, 0.54243, 0.54362]","[0.50592, 0.50389, 0.53039, 0.50194, 0.50111]",0.529810,0.508650,0.021160
8,Ranked Solo,1,"[BiggusDeckus, Unforgivenˉ, hotdogwater17, Bur...","[Decøy, Krult, Warnedjacka, Cidez, CaitUltsJFK]","[0.59686, 0.50596, 0.54444, 0.54494, 0.50194]","[0.50926, 0.70386, 0.50578, 0.50111, 0.4947]",0.538828,0.542942,-0.004114
9,Ranked Solo,1,"[Litemeup, GuGubird, WardInUrMumsBush, SL, Bar...","[Kammr, Rhaastafarian, Warnedjacka, GGmeilemei...","[0.51541, 0.50034, 0.49945, 0.71756, 0.57419]","[0.65613, 0.51724, 0.54453, 0.50268, 0.50111]",0.561390,0.544338,0.017052


In [93]:
site= "https://na.op.gg/ranking/ladder/summonerName=warnedjacka"
hdr = {'User-Agent': 'Mozilla/5.0'}
page = requests.get(site, headers=hdr)
soup = BeautifulSoup(page.text, "html.parser")


In [98]:
ladder = []
for i in range(100):
    ladder.append(soup.find_all(class_="select_summoner ranking-table__cell ranking-table__cell--summoner")[i].get_text().strip())

In [113]:
len(ladder)

100

In [125]:
hundredplayerframe = generateLeagueData(ladder)

In [ ]:
started at 7:43:45
finished 7:46:25

In [107]:
(tenplayerframe[tenplayerframe.Queue_Type == "Ranked Solo"])

,Queue_Type,blue_win,blue_team,red_team,blue_winrates,red_winrates,blue_avg_wr,red_avg_wr,wr_difference
0,Ranked Solo,0,"[Alakayv2, CursedObject, KAYLEOTPBABY, Icysyke...","[potatoporcupine1, Honker, Dingool, FIâsh, Fel...","[0.51432, 0.52778, 0.49391, 0.51838, 0.5113]","[0.52632, 0.60674, 0.53684, 0.50647, 0.5047]",0.513138,0.536214,-0.023076
1,Ranked Solo,1,"[Icysyke, CllT2SI, MyOwnProfile, Kewlipo2, Høà...","[iPav, SOFM1V9HACK, Omly, adcsucks24, FaceChec...","[0.53568, 0.52778, 0.50139, 0.57107, 0.51801]","[0.51701, 0.66942, 0.53525, 0.56218, 0.50198]",0.530786,0.557168,-0.026382
2,Ranked Solo,1,"[veerchand101, SOFM1V9HACK, Aloneus, TheDuola,...","[1v9Cat, Straws, Andresu7, Icysyke, HydroZenith]","[0.51346, 0.51423, 0.66942, 0.50062, 0.52034]","[0.53097, 0.51363, 0.52778, 0.54112, 0.50224]",0.543614,0.523148,0.020466
3,Ranked Solo,1,"[Bigj8, xddar, MySwordCrimson, moveonitsop, TH...","[DJtoilet, BootySIapper, AlexKari, Icysyke, No...","[0.50138, 0.6385, 0.51589, 0.56259, 0.55714]","[0.52778, 0.511, 0.50416, 0.50852, 0.51845]",0.555100,0.513982,0.041118
4,Ranked Solo,1,"[Bigj8, LeokNm, SkyTec, Icysyke, Yoqen]","[Vorever, Naracid, Andresu7, ColtPlaysLeague, ...","[0.58421, 0.52778, 0.51128, 0.51589, 0.59375]","[0.50772, 0.50684, 0.51763, 0.50686, 0.50224]",0.546582,0.508258,0.038324
8,Ranked Solo,1,"[BenBerplanke, YoungTactician, Borozz, Icysyke...","[BiggerEgoWins, JungleEg0, QuoVadıs, Tos8719, ...","[0.51163, 0.51328, 0.51222, 0.52778, 0.48822]","[0.52218, 0.53252, 0.51192, 0.50311, 0.49568]",0.510626,0.513082,-0.002456
9,Ranked Solo,1,"[Heisendong, StixKing, Pan0ptic, minui, MrJune...","[Teemoisweak, WARMACHINEHYPER, GodBranQc27, Ic...","[0.53608, 0.54128, 0.60526, 0.51349, 0.5025]","[0.50712, 0.51656, 0.5092, 0.52778, 0.50448]",0.539722,0.513028,0.026694
0,Ranked Solo,1,"[AhriInari, NinetaIes, alatank, Vergrel, Paige]","[Mangoˉ, Mictest, KingTylerP, shaohekele, Crim...","[0.50283, 0.54913, 0.45631, 0.53945, 0.489]","[0.50487, 0.50566, 0.49777, 0.49474, 0.56805]",0.507344,0.514218,-0.006874
1,Ranked Solo,0,"[GiftedByGods, xSimo, MapSlim, BottledUp, Sojs]","[DippyDan, Miirror, KingTylerP, shaohekele, Ka...","[0.49096, 0.50826, 0.51911, 0.55373, 0.5084]","[0.50969, 0.50787, 0.49777, 0.56805, 0.49664]",0.516092,0.516004,0.000088
2,Ranked Solo,1,"[McGao, vStarzzz, EgirlSupMidRun, Vaalen, MEME...","[Comebox, TLCLightrocket, KingTylerP, shaoheke...","[0.49003, 0.51341, 0.52935, 0.49103, 0.50803]","[0.50742, 0.49777, 0.56013, 0.56805, 0.51737]",0.506370,0.530148,-0.023778


In [111]:
np.mean(tenplayerframe[tenplayerframe.Queue_Type == "Ranked Solo"]["red_avg_wr"])

0.529679655172414

In [118]:
generateLeagueData(["Venerated"])

,Queue_Type,blue_win,blue_team,red_team,blue_winrates,red_winrates,blue_avg_wr,red_avg_wr,wr_difference
0,URF,1,"[LargeTestSickle, onkyoh, Venerated, CuIe, Pan...","[OriginalYauker, NATECOOL25, ImKiingDarky115, ...","[0.54355, 0.50314, 0.38095, 0.5, 0.5]","[0.52758, 0.50476, 0.5116, 0.44604, 0.5]",0.485528,0.497996,-0.012468
1,URF,1,"[LargeTestSickle, onkyoh, Venerated, CuIe, Nob...","[ColourlessGreen1, zeekzun, PabloShacon, LokiT...","[0.54355, 0.50314, 0.38095, 0.51429, 0.5]","[0.50888, 0.5, 0.5, 0.5, 0.5]",0.488386,0.501776,-0.013390
2,Normal,1,"[Venerated, onkyoh, CuIe, LargeTestSickle, Med...","[achowii, jagrosh, SlyPermaBan5, Cathulhu, bir...","[0.54355, 0.50314, 0.38095, 0.5, 0.5]","[0.51929, 0.61765, 0.52466, 0.52842, 0.5]",0.485528,0.538004,-0.052476
3,Normal,0,"[MissBoomBox25, ThëReaper, NGAVoid, Leogic, ju...","[CuIe, onkyoh, Venerated, MedicinalHentai, Lar...","[0.4539, 0.4, 0.49625, 0.37864, 0.43434]","[0.54355, 0.50314, 0.38095, 0.5, 0.5]",0.432626,0.485528,-0.052902
4,URF,1,"[jacksss233, IScream, gimmethetimmy, LOVEU3000...","[Coisini, onkyoh, Venerated, CoconutMilkTea, C...","[0.48347, 0.53153, 0.49826, 0.48624, 0.53543]","[0.47742, 0.54355, 0.50314, 0.38095, 0.5]",0.506986,0.481012,0.025974
5,Normal,1,"[LittIecMoBn, onkyoh, CuIe, Venerated, LargeTe...","[enrutcoN, AshuPrimu, JhinJoker, Camous, Dayne...","[0.54355, 0.50314, 0.38095, 0.5, 0.5]","[0.5567, 0.53125, 0.508, 0.51724, 0.5]",0.485528,0.522638,-0.037110
6,URF,0,"[No1ButtPirate, FancyManlyBeard, BÀbuFrik, Phe...","[LargeTestSickle, Daretoplay, onkyoh, Venerate...","[0.51429, 0.47674, 0.5, 0.5, 0.5]","[0.54355, 0.50314, 0.38095, 0.36364, 0.5]",0.498206,0.458256,0.039950
7,Normal,0,"[monDVster, LunasLostWarrior, bonbl, PawgyWawg...","[LargeTestSickle, onkyoh, Venerated, ElasticBu...","[0.58182, 0.54839, 0.49206, 0.5, 0.5]","[0.54355, 0.50314, 0.38095, 0.5, 0.5]",0.524454,0.485528,0.038926
8,Normal,0,"[CuIe, TUGOverlordzelo, Venerated, onkyoh, Lar...","[Bru, TuturuLady, Langoo, WetDream6, AverageGa...","[0.54355, 0.50314, 0.51136, 0.38095, 0.5]","[0.50526, 0.52195, 0.5303, 0.4661, 0.5]",0.487800,0.504722,-0.016922
9,Normal,0,"[YesUhhuhYea, Datchi, ReyisBae, Flipadelphia, ...","[onkyoh, gouphigh, Venerated, CuIe, LargeTestS...","[0.57736, 0.50968, 0.5, 0.5, 0.5]","[0.54355, 0.50314, 0.38095, 0.5, 0.5]",0.517408,0.485528,0.031880


In [143]:
np.mean(hundredplayerframe[(hundredplayerframe.Queue_Type == "Ranked Solo") & ((hundredplayerframe.blue_win == 0))]["wr_difference"])*100

-0.38224444444444505

In [136]:
hundredplayerframe[(hundredplayerframe.Queue_Type == "Ranked Solo") & (hundredplayerframe.blue_win == 1)][]

,Queue_Type,blue_win,blue_team,red_team,blue_winrates,red_winrates,blue_avg_wr,red_avg_wr,wr_difference
1,Ranked Solo,1,"[Icysyke, CllT2SI, MyOwnProfile, Kewlipo2, Høà...","[iPav, SOFM1V9HACK, Omly, adcsucks24, FaceChec...","[0.53568, 0.52778, 0.50139, 0.56962, 0.51801]","[0.51701, 0.66942, 0.53525, 0.56218, 0.50198]",0.530496,0.557168,-0.026672
2,Ranked Solo,1,"[veerchand101, SOFM1V9HACK, Aloneus, TheDuola,...","[1v9Cat, Straws, Andresu7, Icysyke, HydroZenith]","[0.51346, 0.51423, 0.66942, 0.50062, 0.52034]","[0.53004, 0.51363, 0.52778, 0.54187, 0.50224]",0.543614,0.523112,0.020502
3,Ranked Solo,1,"[Bigj8, xddar, MySwordCrimson, moveonitsop, TH...","[DJtoilet, BootySIapper, AlexKari, Icysyke, No...","[0.50138, 0.6385, 0.51493, 0.56259, 0.55714]","[0.52778, 0.511, 0.50444, 0.50852, 0.51839]",0.554908,0.514026,0.040882
4,Ranked Solo,1,"[Bigj8, LeokNm, SkyTec, Icysyke, Yoqen]","[Vorever, Naracid, Andresu7, ColtPlaysLeague, ...","[0.58421, 0.52778, 0.51128, 0.51493, 0.59375]","[0.50772, 0.50684, 0.51763, 0.50686, 0.50224]",0.546390,0.508258,0.038132
8,Ranked Solo,1,"[BenBerplanke, YoungTactician, Borozz, Icysyke...","[BiggerEgoWins, JungleEg0, QuoVadıs, Tos8719, ...","[0.51163, 0.51328, 0.51222, 0.52778, 0.48822]","[0.52218, 0.53252, 0.51235, 0.50363, 0.49532]",0.510626,0.513200,-0.002574
...,...,...,...,...,...,...,...,...,...
7,Ranked Solo,1,"[Gooonyou, FlandreBot, HBX11, madaphrobuny, Dr...","[merrjerry, DarkStarShaco, lovies16, Chronoadc...","[0.52252, 0.51227, 0.63415, 0.53004, 0.49723]","[0.52805, 0.56032, 0.51119, 0.64368, 0.50922]",0.539242,0.550492,-0.011250
9,Ranked Solo,1,"[ƒeederŃμmþerº1, DandelionYi, imaqtkat, Puerto...","[Gooonyou, OrgasmicP, MIANARE, ShoTiFizE, Your...","[0.50919, 0.51544, 0.53412, 0.51989, 0.51997]","[0.50074, 0.51227, 0.50257, 0.5, 0.49645]",0.519722,0.502406,0.017316
1,Ranked Solo,1,"[Aerodox, 7707, DisneyBambi, EgoDestroyer, Sor...","[TroyJK, PrincessBlue, Antoine1234567, buhuiwa...","[0.52255, 0.50172, 0.625, 0.50642, 0.55791]","[0.51098, 0.54386, 0.52449, 0.51043, 0.49863]",0.542720,0.517678,0.025042
5,Ranked Solo,1,"[AngelsDemon, Morol, TaeSo3o, Int4Lyfe, TeetoJ...","[stanlasting, 7707, DisneyBambi, taigon1, cris...","[0.49409, 0.51389, 0.50957, 0.51455, 0.54506]","[0.52255, 0.50182, 0.57143, 0.50844, 0.55791]",0.515432,0.532430,-0.016998


In [146]:
from sklearn.linear_model import LogisticRegression

In [147]:
X = hundredplayerframe["blue_avg_wr"]
y = hundredplayerframe["blue_win"]
clf = LogisticRegression(random_state=0).fit(X, y)

ValueError: Expected 2D array, got 1D array instead:
array=[0.513382 0.530496 0.543614 0.554908 0.54639  0.49527  0.487962 0.523788
 0.510626 0.539572 0.507186 0.515674 0.50637  0.518078 0.532534 0.51394
 0.53655  0.519084 0.529868 0.585314 0.505112 0.526432 0.535288 0.533634
 0.511516 0.51447  0.536442 0.511268 0.495056 0.532364 0.502756 0.521156
 0.500262 0.521156 0.508032 0.54975  0.552328 0.552328 0.531032 0.546612
 0.511718 0.518338 0.502598 0.517432 0.525072 0.536822 0.508464 0.493508
 0.528708 0.523814 0.566804 0.525102 0.548712 0.52266  0.516546 0.534606
 0.536028 0.50738  0.51117  0.542924 0.52998  0.518146 0.559442 0.531408
 0.531784 0.529378 0.51703  0.511836 0.542022 0.522944 0.511836 0.529316
 0.533044 0.510772 0.538376 0.51229  0.509028 0.5715   0.580434 0.519202
 0.517716 0.532304 0.54177  0.50608  0.543952 0.51545  0.555472 0.541798
 0.526384 0.506236 0.506278 0.49864  0.482558 0.500002 0.48845  0.516198
 0.483296 0.478442 0.488778 0.494818 0.53239  0.489252 0.485922 0.

In [148]:
finalframe = hundredplayerframe[hundredplayerframe.Queue_Type == "Ranked Solo"]
output = finalframe[["blue_win", "blue_avg_wr", "red_avg_wr"]]

In [149]:
output

,blue_win,blue_avg_wr,red_avg_wr
0,0,0.513382,0.536298
1,1,0.530496,0.557168
2,1,0.543614,0.523112
3,1,0.554908,0.514026
4,1,0.546390,0.508258
...,...,...,...
5,1,0.515432,0.532430
6,0,0.531570,0.547918
7,0,0.528192,0.551196
8,0,0.512760,0.537172


In [151]:
output.to_csv("league_wr_data.csv")